In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import  StandardScaler, FunctionTransformer, OneHotEncoder

In [18]:
df=pd.read_csv("data/get_around_pricing_project.csv")
df.drop(columns=["Unnamed: 0"], inplace=True) # Suppression de la colonne Unamed: 
df.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [19]:
df_num=df.select_dtypes(exclude="object")
df_num.head()

,mileage,engine_power,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,140411,100,True,True,False,False,True,True,True,106
1,13929,317,True,True,False,False,False,True,True,264
2,183297,120,False,False,False,False,True,False,True,101
3,128035,135,True,True,False,False,True,True,True,158
4,97097,160,True,True,False,False,False,True,True,183


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   model_key                  4843 non-null   object
 1   mileage                    4843 non-null   int64 
 2   engine_power               4843 non-null   int64 
 3   fuel                       4843 non-null   object
 4   paint_color                4843 non-null   object
 5   car_type                   4843 non-null   object
 6   private_parking_available  4843 non-null   bool  
 7   has_gps                    4843 non-null   bool  
 8   has_air_conditioning       4843 non-null   bool  
 9   automatic_car              4843 non-null   bool  
 10  has_getaround_connect      4843 non-null   bool  
 11  has_speed_regulator        4843 non-null   bool  
 12  winter_tires               4843 non-null   bool  
 13  rental_price_per_day       4843 non-null   int64 
dtypes: bool(

In [21]:
df_num.columns

Index(['mileage', 'engine_power', 'private_parking_available', 'has_gps',
       'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
       'has_speed_regulator', 'winter_tires', 'rental_price_per_day'],
      dtype='object')

# Filtre les données inconscistantes

In [22]:
df = df[(df['mileage'] >= 0) & (df['engine_power'] > 0) & (df['rental_price_per_day'] > 0)]

In [23]:
X = df.drop("rental_price_per_day", axis=1)
Y = df["rental_price_per_day"]

In [24]:
numerics = []
booleans = []
categorics = []

for i, type in X.dtypes.items():
    type = str(type)
    if ('float' in type) or ('int' in type):
        numerics.append(i)
    elif ('bool' in type):
        booleans.append(i)
    else:
        categorics.append(i)

print(numerics)
print(booleans)
print(categorics)

['mileage', 'engine_power']
['private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
['model_key', 'fuel', 'paint_color', 'car_type']


In [25]:
for category in categorics:
    fig = px.histogram(df, x=category, y='rental_price_per_day', histfunc="avg", marginal="histogram")
    fig.show()

#Visualisation des inégalités de la répartition des données

In [26]:
for category in categorics:
    fig = px.histogram(df, x=category)
    fig.update_xaxes(categoryorder="total descending")
    fig.show()

In [27]:
X = df.drop("rental_price_per_day", axis = 1)

# Tranformation des valeurs catégorielles et booléennes en valeurs numériques.
#Standardisation des valeurs numériques

In [28]:
ohe = OneHotEncoder(drop=None, handle_unknown="error", sparse_output=False)
stdscaler = StandardScaler()
ftn_transform = FunctionTransformer(None, feature_names_out='one-to-one')  # Regarder feature_names_out

processor = ColumnTransformer(
    transformers=[
        ("categorics", ohe, categorics),
        ("numerics", stdscaler, numerics),
        ("booleans", ftn_transform, booleans)
    ]
)

X_preprocessed = processor.fit_transform(X)

AttributeError: 'StandardScaler' object has no attribute 'mean_'

In [34]:
stdscaler.fit(df[["engine_power"]])
print(stdscaler.mean_)
print(stdscaler.scale_)

[128.9940095]
[38.92623235]


In [ ]:
new_labels = []
for feature in processor.get_feature_names_out():
    feature = feature.replace("categorics__", "").replace("numerics__", "").replace("booleans__", "")
    new_labels.append(feature)

new_labels

['model_key_Alfa Romeo',
 'model_key_Audi',
 'model_key_BMW',
 'model_key_Citroën',
 'model_key_Ferrari',
 'model_key_Fiat',
 'model_key_Ford',
 'model_key_Honda',
 'model_key_KIA Motors',
 'model_key_Lamborghini',
 'model_key_Lexus',
 'model_key_Maserati',
 'model_key_Mazda',
 'model_key_Mercedes',
 'model_key_Mini',
 'model_key_Mitsubishi',
 'model_key_Nissan',
 'model_key_Opel',
 'model_key_PGO',
 'model_key_Peugeot',
 'model_key_Porsche',
 'model_key_Renault',
 'model_key_SEAT',
 'model_key_Subaru',
 'model_key_Suzuki',
 'model_key_Toyota',
 'model_key_Volkswagen',
 'model_key_Yamaha',
 'fuel_diesel',
 'fuel_electro',
 'fuel_hybrid_petrol',
 'fuel_petrol',
 'paint_color_beige',
 'paint_color_black',
 'paint_color_blue',
 'paint_color_brown',
 'paint_color_green',
 'paint_color_grey',
 'paint_color_orange',
 'paint_color_red',
 'paint_color_silver',
 'paint_color_white',
 'car_type_convertible',
 'car_type_coupe',
 'car_type_estate',
 'car_type_hatchback',
 'car_type_sedan',
 'car_t

# Création d'un nouveau Dataframe avec les nouvelles valeurs catégorielles
#Preprocessing des données

In [ ]:
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=new_labels, index=df.index)
X_preprocessed_df["rental_price_per_day"] = df["rental_price_per_day"]

#Export de notre Dataframe en csv

In [ ]:
X_preprocessed_df.to_csv("data/preprocessed.csv")